In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from tqdm import tqdm
import time

train = pd.read_csv("CSV_train.csv",low_memory=False,delimiter=';')
test=pd.read_csv("CSV_test.csv",low_memory=False,delimiter=',')
hidden=pd.read_csv("CSV_hidden_test.csv",low_memory=False,delimiter=',')

In [2]:
# storing length of datasets 
train_len = train.shape[0] 
test_len = test.shape[0]
All_data = pd.concat((train,test,hidden)).reset_index(drop=True) 

lithology_keys = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}
All_data['Lithology'] = All_data['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_keys)
All_data

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE,Lithology
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0,Shale
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0,Shale
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0,Shale
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0,Shale
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0,Shale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,136.911575,NaN,0.502458,NaN,2.311106,24.306124,NaN,65000,2.0,Shale
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,137.583923,NaN,0.374753,NaN,1.853418,22.201078,NaN,65000,2.0,Shale
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,138.310898,NaN,0.211487,NaN,1.325961,20.096741,NaN,65000,2.0,Shale
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,137.592819,NaN,0.147950,NaN,1.260347,17.992323,NaN,65000,2.0,Shale


In [3]:
#dropping columns with high missing values
drop_cols = ['SGR', 'ROPA', 'RXO', 'MUDWEIGHT','DCAL','RMIC','FORCE_2020_LITHOFACIES_CONFIDENCE']
All_data_drop = All_data.drop(drop_cols, axis=1)
# encoding categorical variables
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP'].astype('category')
All_data_drop['GROUP_encoded'] = All_data_drop['GROUP_encoded'].cat.codes

All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION'].astype('category')
All_data_drop['FORMATION_encoded'] = All_data_drop['FORMATION_encoded'].cat.codes

All_data_drop['WELL_encoded'] = All_data_drop['WELL'].astype('category')
All_data_drop['WELL_encoded'] = All_data_drop['WELL_encoded'].cat.codes

All_data_drop['Lithology_encoded'] = All_data_drop['FORCE_2020_LITHOFACIES_LITHOLOGY'].astype('category')
All_data_drop['Lithology_encoded'] = All_data_drop['Lithology_encoded'].cat.codes

In [4]:
All_data_drop

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,BS,ROP,DTS,DRHO,FORCE_2020_LITHOFACIES_LITHOLOGY,Lithology,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,15/9-13,494.5280,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,NaN,34.636410,NaN,-0.574928,65000,Shale,6,-1,1,1
1,15/9-13,494.6800,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,NaN,34.636410,NaN,-0.570188,65000,Shale,6,-1,1,1
2,15/9-13,494.8320,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,NaN,34.779556,NaN,-0.574245,65000,Shale,6,-1,1,1
3,15/9-13,494.9840,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,NaN,39.965164,NaN,-0.586315,65000,Shale,6,-1,1,1
4,15/9-13,495.1360,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,NaN,57.483765,NaN,-0.597914,65000,Shale,6,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,35/9-7,2973.2988,536096.06250,6793022.0,-2943.444580,BAAT GP.,Etive Fm.,8.276272,NaN,2.820439,...,8.5,15.195305,136.911575,0.502458,65000,Shale,0,16,113,1
1429690,35/9-7,2973.4508,536096.06250,6793022.0,-2943.595947,BAAT GP.,Etive Fm.,8.267273,NaN,3.020778,...,8.5,15.770223,137.583923,0.374753,65000,Shale,0,16,113,1
1429691,35/9-7,2973.6028,536096.06250,6793022.0,-2943.747559,BAAT GP.,Etive Fm.,8.250099,NaN,2.795711,...,8.5,16.418465,138.310898,0.211487,65000,Shale,0,16,113,1
1429692,35/9-7,2973.7548,536096.06250,6793022.0,-2943.899170,BAAT GP.,Etive Fm.,NaN,NaN,2.658694,...,8.5,17.037945,137.592819,0.147950,65000,Shale,0,16,113,1


In [5]:
#dropping categorial features replaces beforehan by encoded features
drop2 = All_data_drop.drop(['GROUP', 'FORMATION','WELL','FORCE_2020_LITHOFACIES_LITHOLOGY','Lithology'], axis=1)

# splitting dataset into training, test, and hidden sets
train_prep = drop2[:train_len].copy()
test_prep = drop2[train_len:(train_len+test_len)].copy()
hidden_prep = drop2[(train_len+test_len):].copy()

In [6]:
drop2

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,NaN,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,NaN,34.636410,NaN,-0.574928,6,-1,1,1
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,NaN,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,NaN,34.636410,NaN,-0.570188,6,-1,1,1
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,NaN,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,NaN,34.779556,NaN,-0.574245,6,-1,1,1
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,NaN,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,NaN,39.965164,NaN,-0.586315,6,-1,1,1
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,NaN,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,NaN,57.483765,NaN,-0.597914,6,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,NaN,2.820439,3.158570,NaN,90.720284,...,75.260658,NaN,8.5,15.195305,136.911575,0.502458,0,16,113,1
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,NaN,3.020778,3.332977,NaN,87.062027,...,74.868301,NaN,8.5,15.770223,137.583923,0.374753,0,16,113,1
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,NaN,2.795711,3.044179,NaN,86.115921,...,74.848122,NaN,8.5,16.418465,138.310898,0.211487,0,16,113,1
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,NaN,NaN,2.658694,2.847681,NaN,89.497131,...,74.964027,NaN,8.5,17.037945,137.592819,0.147950,0,16,113,1


In [7]:
train_prep1= train_prep.copy()
test_prep1= test_prep.copy()
hidden_prep1= hidden_prep.copy()

In [8]:
#Inputing missing values by introducing median 
from sklearn.impute import SimpleImputer
miss = SimpleImputer(missing_values=np.nan, strategy='median')
miss.fit(drop2)
All_imp = miss.fit_transform(drop2)
All_imp=pd.DataFrame(All_imp, columns=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO',
         'GROUP_encoded',
       'FORMATION_encoded', 'WELL_encoded','Lithology_encoded'])
All_imp

,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,CALI,RSHA,RMED,RDEP,RHOB,GR,...,DTC,SP,BS,ROP,DTS,DRHO,GROUP_encoded,FORMATION_encoded,WELL_encoded,Lithology_encoded
0,494.5280,437641.96875,6470972.5,-469.501831,19.480835,1.398049,1.611410,1.798681,1.884186,80.200851,...,161.131180,24.612379,12.250001,34.636410,189.362198,-0.574928,6.0,-1.0,1.0,1.0
1,494.6800,437641.96875,6470972.5,-469.653809,19.468800,1.398049,1.618070,1.795641,1.889794,79.262886,...,160.603470,23.895531,12.250001,34.636410,189.362198,-0.570188,6.0,-1.0,1.0,1.0
2,494.8320,437641.96875,6470972.5,-469.805786,19.468800,1.398049,1.626459,1.800733,1.896523,74.821999,...,160.173615,23.916357,12.250001,34.779556,189.362198,-0.574245,6.0,-1.0,1.0,1.0
3,494.9840,437641.96875,6470972.5,-469.957794,19.459282,1.398049,1.621594,1.801517,1.891913,72.878922,...,160.149429,23.793688,12.250001,39.965164,189.362198,-0.586315,6.0,-1.0,1.0,1.0
4,495.1360,437641.96875,6470972.5,-470.109772,19.453100,1.398049,1.602679,1.795299,1.880034,71.729141,...,160.128342,24.104078,12.250001,57.483765,189.362198,-0.597914,6.0,-1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429689,2973.2988,536096.06250,6793022.0,-2943.444580,8.276272,1.398049,2.820439,3.158570,2.331407,90.720284,...,75.260658,54.270451,8.500000,15.195305,136.911575,0.502458,0.0,16.0,113.0,1.0
1429690,2973.4508,536096.06250,6793022.0,-2943.595947,8.267273,1.398049,3.020778,3.332977,2.331407,87.062027,...,74.868301,54.270451,8.500000,15.770223,137.583923,0.374753,0.0,16.0,113.0,1.0
1429691,2973.6028,536096.06250,6793022.0,-2943.747559,8.250099,1.398049,2.795711,3.044179,2.331407,86.115921,...,74.848122,54.270451,8.500000,16.418465,138.310898,0.211487,0.0,16.0,113.0,1.0
1429692,2973.7548,536096.06250,6793022.0,-2943.899170,12.515673,1.398049,2.658694,2.847681,2.331407,89.497131,...,74.964027,54.270451,8.500000,17.037945,137.592819,0.147950,0.0,16.0,113.0,1.0


In [9]:
from sklearn.svm import OneClassSVM
svm = OneClassSVM(nu=0.1)
yhat = svm.fit_predict(All_imp)
mask = yhat != -1
train_data_svm = All_imp[mask]

In [10]:
train_imp = train_data_svm[:train_len].copy()
test_imp = train_data_svm[train_len:(train_len+test_len)].copy()
hidden_imp = train_data_svm[(train_len+test_len):].copy()

In [11]:
print(train_imp.shape)
print(test_imp.shape)
print(hidden_imp.shape)

(1170511, 22)
(116087, 22)
(0, 22)


In [13]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
x_header=['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RSHA', 'RMED', 'RDEP',
       'RHOB', 'GR', 'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DRHO',
       'GROUP_encoded', 'FORMATION_encoded', 'WELL_encoded']
y_header=['Lithology_encoded']
x_train = train_imp[x_header]
y_train = train_imp[y_header]
x_test = test_imp[x_header]
y_test = test_imp[y_header]
x_hidden = hidden_imp[x_header]
y_hidden = hidden_imp[y_header]

##Min-Max scaler 
scaler = MinMaxScaler()
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()
#x_hidden_scaled = x_hidden.copy()

x_train_scaled.iloc[:,:18] = scaler.fit_transform(x_train_scaled.iloc[:,:18])
x_test_scaled.iloc[:,:18] = scaler.transform(x_test_scaled.iloc[:,:18])
#x_hidden_scaled.iloc[:,:18] = scaler.transform(x_hidden_scaled.iloc[:,:18])

In [14]:
#Supervised Algorithms
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsRegressor
from pprint import pprint
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
#Comparing base models accuracies by using k-fold cross validation - 10 folds

from sklearn.model_selection import cross_val_score

new_train = pd.concat((x_train_scaled, pd.DataFrame(y_train, columns=["Lithology_encoded"])), axis=1)

#Randomly sampling data
sampled_train = new_train.sample(n=100000, random_state=0)


#Spliting training data
x_train_sam = sampled_train.drop(["Lithology_encoded"], axis=1)
y_train_sam = sampled_train["Lithology_encoded"]



C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [15]:
estimator = LogisticRegression(C=1e-3, solver='saga', max_iter=4000)
score = cross_val_score(estimator, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score

0.4750526391586196

In [16]:
estimator1 = DecisionTreeClassifier()
score = cross_val_score(estimator1, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score

0.8622563892587005

In [17]:
estimator2 = RandomForestClassifier()
score = cross_val_score(estimator2, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score

0.9124031003318412

In [18]:
estimator3 = XGBClassifier()
score3 = cross_val_score(estimator3, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score3

C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:32:05] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:35:21] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


...

C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:55:50] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Ayori\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[01:58:27] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8938595839693428

In [19]:
estimator4 = GradientBoostingClassifier()
score4 = cross_val_score(estimator4, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score4


0.8312794054230999

In [20]:
from sklearn.neighbors import KNeighborsClassifier
estimator5 =KNeighborsClassifier()
score5 = cross_val_score(estimator5, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score5


C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 197, in predict
    neigh_dist, neigh_ind 

C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 197, in predict
    neigh_dist, neigh_ind 

C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\Ayori\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py", line 197, in predict
    neigh_dist, neigh_ind 

nan

In [21]:

estimator6 =SVC()
score6 = cross_val_score(estimator6, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score6


0.47440698043711044

In [22]:
from catboost import CatBoostClassifier
estimator7 =CatBoostClassifier()
score7 = cross_val_score(estimator7, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score7


Learning rate set to 0.099538
0:	learn: 1.9461250	total: 442ms	remaining: 7m 21s
1:	learn: 1.7090498	total: 739ms	remaining: 6m 8s
2:	learn: 1.5496148	total: 1.05s	remaining: 5m 48s
3:	learn: 1.4224466	total: 1.33s	remaining: 5m 31s
4:	learn: 1.3272651	total: 1.61s	remaining: 5m 20s
5:	learn: 1.2482990	total: 1.89s	remaining: 5m 12s
6:	learn: 1.1834247	total: 2.14s	remaining: 5m 3s
7:	learn: 1.1299732	total: 2.38s	remaining: 4m 55s
8:	learn: 1.0835438	total: 2.64s	remaining: 4m 50s
9:	learn: 1.0431660	total: 2.91s	remaining: 4m 48s
10:	learn: 1.0034672	total: 3.33s	remaining: 4m 59s
11:	learn: 0.9686799	total: 3.67s	remaining: 5m 2s
12:	learn: 0.9402455	total: 3.97s	remaining: 5m 1s
13:	learn: 0.9134862	total: 4.21s	remaining: 4m 56s
14:	learn: 0.8896300	total: 4.44s	remaining: 4m 51s
15:	learn: 0.8643297	total: 4.69s	remaining: 4m 48s
16:	learn: 0.8440536	total: 4.92s	remaining: 4m 44s
17:	learn: 0.8261356	total: 5.18s	remaining: 4m 42s
18:	learn: 0.8116673	total: 5.45s	remaining: 4m 

158:	learn: 0.4309167	total: 47.2s	remaining: 4m 9s
159:	learn: 0.4302391	total: 47.5s	remaining: 4m 9s
160:	learn: 0.4295149	total: 47.9s	remaining: 4m 9s
161:	learn: 0.4288323	total: 48.2s	remaining: 4m 9s
162:	learn: 0.4280512	total: 48.6s	remaining: 4m 9s
163:	learn: 0.4276672	total: 48.9s	remaining: 4m 9s
164:	learn: 0.4270631	total: 49.2s	remaining: 4m 9s
165:	learn: 0.4267076	total: 49.5s	remaining: 4m 8s
166:	learn: 0.4259996	total: 49.9s	remaining: 4m 8s
167:	learn: 0.4256859	total: 50.2s	remaining: 4m 8s
168:	learn: 0.4253194	total: 50.6s	remaining: 4m 8s
169:	learn: 0.4245634	total: 50.9s	remaining: 4m 8s
170:	learn: 0.4238918	total: 51.3s	remaining: 4m 8s
171:	learn: 0.4232587	total: 51.6s	remaining: 4m 8s
172:	learn: 0.4228167	total: 51.9s	remaining: 4m 8s
173:	learn: 0.4219787	total: 52.3s	remaining: 4m 8s
174:	learn: 0.4214308	total: 52.6s	remaining: 4m 7s
175:	learn: 0.4206056	total: 52.9s	remaining: 4m 7s
176:	learn: 0.4200433	total: 53.3s	remaining: 4m 7s
177:	learn: 

313:	learn: 0.3595786	total: 1m 40s	remaining: 3m 39s
314:	learn: 0.3591042	total: 1m 40s	remaining: 3m 39s
315:	learn: 0.3586506	total: 1m 41s	remaining: 3m 39s
316:	learn: 0.3582561	total: 1m 41s	remaining: 3m 38s
317:	learn: 0.3579926	total: 1m 41s	remaining: 3m 38s
318:	learn: 0.3576520	total: 1m 42s	remaining: 3m 38s
319:	learn: 0.3573728	total: 1m 42s	remaining: 3m 38s
320:	learn: 0.3572032	total: 1m 43s	remaining: 3m 37s
321:	learn: 0.3569978	total: 1m 43s	remaining: 3m 37s
322:	learn: 0.3566512	total: 1m 43s	remaining: 3m 37s
323:	learn: 0.3564383	total: 1m 44s	remaining: 3m 37s
324:	learn: 0.3560829	total: 1m 44s	remaining: 3m 37s
325:	learn: 0.3557980	total: 1m 45s	remaining: 3m 37s
326:	learn: 0.3555966	total: 1m 45s	remaining: 3m 37s
327:	learn: 0.3553705	total: 1m 45s	remaining: 3m 36s
328:	learn: 0.3550133	total: 1m 46s	remaining: 3m 36s
329:	learn: 0.3547944	total: 1m 46s	remaining: 3m 36s
330:	learn: 0.3545284	total: 1m 46s	remaining: 3m 35s
331:	learn: 0.3542704	total:

467:	learn: 0.3234525	total: 2m 32s	remaining: 2m 53s
468:	learn: 0.3233145	total: 2m 33s	remaining: 2m 53s
469:	learn: 0.3231681	total: 2m 33s	remaining: 2m 53s
470:	learn: 0.3229242	total: 2m 33s	remaining: 2m 52s
471:	learn: 0.3227386	total: 2m 34s	remaining: 2m 52s
472:	learn: 0.3225326	total: 2m 34s	remaining: 2m 52s
473:	learn: 0.3223370	total: 2m 34s	remaining: 2m 51s
474:	learn: 0.3219784	total: 2m 35s	remaining: 2m 51s
475:	learn: 0.3214710	total: 2m 35s	remaining: 2m 51s
476:	learn: 0.3213348	total: 2m 35s	remaining: 2m 50s
477:	learn: 0.3211444	total: 2m 36s	remaining: 2m 50s
478:	learn: 0.3210212	total: 2m 36s	remaining: 2m 50s
479:	learn: 0.3207817	total: 2m 36s	remaining: 2m 50s
480:	learn: 0.3205510	total: 2m 37s	remaining: 2m 49s
481:	learn: 0.3202573	total: 2m 37s	remaining: 2m 49s
482:	learn: 0.3201085	total: 2m 38s	remaining: 2m 49s
483:	learn: 0.3198806	total: 2m 38s	remaining: 2m 48s
484:	learn: 0.3197784	total: 2m 38s	remaining: 2m 48s
485:	learn: 0.3196346	total:

620:	learn: 0.2987554	total: 3m 24s	remaining: 2m 4s
621:	learn: 0.2986440	total: 3m 24s	remaining: 2m 4s
622:	learn: 0.2985806	total: 3m 25s	remaining: 2m 4s
623:	learn: 0.2984723	total: 3m 25s	remaining: 2m 3s
624:	learn: 0.2983869	total: 3m 25s	remaining: 2m 3s
625:	learn: 0.2983165	total: 3m 26s	remaining: 2m 3s
626:	learn: 0.2981440	total: 3m 26s	remaining: 2m 2s
627:	learn: 0.2979825	total: 3m 26s	remaining: 2m 2s
628:	learn: 0.2979243	total: 3m 27s	remaining: 2m 2s
629:	learn: 0.2977870	total: 3m 27s	remaining: 2m 1s
630:	learn: 0.2975077	total: 3m 27s	remaining: 2m 1s
631:	learn: 0.2974451	total: 3m 28s	remaining: 2m 1s
632:	learn: 0.2972931	total: 3m 28s	remaining: 2m
633:	learn: 0.2971584	total: 3m 28s	remaining: 2m
634:	learn: 0.2970604	total: 3m 29s	remaining: 2m
635:	learn: 0.2969559	total: 3m 29s	remaining: 1m 59s
636:	learn: 0.2967387	total: 3m 29s	remaining: 1m 59s
637:	learn: 0.2966152	total: 3m 30s	remaining: 1m 59s
638:	learn: 0.2965164	total: 3m 30s	remaining: 1m 58

...

442:	learn: 0.3297149	total: 2m 25s	remaining: 3m 3s
443:	learn: 0.3296092	total: 2m 26s	remaining: 3m 3s
444:	learn: 0.3292499	total: 2m 26s	remaining: 3m 2s
445:	learn: 0.3289372	total: 2m 26s	remaining: 3m 2s
446:	learn: 0.3286195	total: 2m 27s	remaining: 3m 2s
447:	learn: 0.3283968	total: 2m 27s	remaining: 3m 1s
448:	learn: 0.3280425	total: 2m 27s	remaining: 3m 1s
449:	learn: 0.3278397	total: 2m 28s	remaining: 3m 1s
450:	learn: 0.3276527	total: 2m 28s	remaining: 3m
451:	learn: 0.3274588	total: 2m 29s	remaining: 3m
452:	learn: 0.3272073	total: 2m 29s	remaining: 3m
453:	learn: 0.3268152	total: 2m 29s	remaining: 3m
454:	learn: 0.3266707	total: 2m 30s	remaining: 3m
455:	learn: 0.3264999	total: 2m 30s	remaining: 2m 59s
456:	learn: 0.3264419	total: 2m 30s	remaining: 2m 59s
457:	learn: 0.3262814	total: 2m 31s	remaining: 2m 59s
458:	learn: 0.3260516	total: 2m 31s	remaining: 2m 58s
459:	learn: 0.3259713	total: 2m 32s	remaining: 2m 58s
460:	learn: 0.3257850	total: 2m 32s	remaining: 2m 58s
46

595:	learn: 0.3029157	total: 3m 16s	remaining: 2m 13s
596:	learn: 0.3027548	total: 3m 17s	remaining: 2m 13s
597:	learn: 0.3026642	total: 3m 17s	remaining: 2m 12s
598:	learn: 0.3024478	total: 3m 17s	remaining: 2m 12s
599:	learn: 0.3023498	total: 3m 18s	remaining: 2m 12s
600:	learn: 0.3022009	total: 3m 18s	remaining: 2m 11s
601:	learn: 0.3020941	total: 3m 18s	remaining: 2m 11s
602:	learn: 0.3019789	total: 3m 19s	remaining: 2m 11s
603:	learn: 0.3018207	total: 3m 19s	remaining: 2m 10s
604:	learn: 0.3016650	total: 3m 19s	remaining: 2m 10s
605:	learn: 0.3014390	total: 3m 20s	remaining: 2m 10s
606:	learn: 0.3013843	total: 3m 20s	remaining: 2m 9s
607:	learn: 0.3011309	total: 3m 20s	remaining: 2m 9s
608:	learn: 0.3008885	total: 3m 21s	remaining: 2m 9s
609:	learn: 0.3007883	total: 3m 21s	remaining: 2m 8s
610:	learn: 0.3005318	total: 3m 21s	remaining: 2m 8s
611:	learn: 0.3004148	total: 3m 22s	remaining: 2m 8s
612:	learn: 0.3002346	total: 3m 22s	remaining: 2m 7s
613:	learn: 0.3001264	total: 3m 22s

748:	learn: 0.2833289	total: 4m 7s	remaining: 1m 22s
749:	learn: 0.2832139	total: 4m 7s	remaining: 1m 22s
750:	learn: 0.2831330	total: 4m 7s	remaining: 1m 22s
751:	learn: 0.2830347	total: 4m 8s	remaining: 1m 21s
752:	learn: 0.2827597	total: 4m 8s	remaining: 1m 21s
753:	learn: 0.2824573	total: 4m 8s	remaining: 1m 21s
754:	learn: 0.2823956	total: 4m 9s	remaining: 1m 20s
755:	learn: 0.2823259	total: 4m 9s	remaining: 1m 20s
756:	learn: 0.2821886	total: 4m 9s	remaining: 1m 20s
757:	learn: 0.2821093	total: 4m 10s	remaining: 1m 19s
758:	learn: 0.2820173	total: 4m 10s	remaining: 1m 19s
759:	learn: 0.2819000	total: 4m 10s	remaining: 1m 19s
760:	learn: 0.2817728	total: 4m 11s	remaining: 1m 18s
761:	learn: 0.2816588	total: 4m 11s	remaining: 1m 18s
762:	learn: 0.2815703	total: 4m 11s	remaining: 1m 18s
763:	learn: 0.2814779	total: 4m 12s	remaining: 1m 17s
764:	learn: 0.2814048	total: 4m 12s	remaining: 1m 17s
765:	learn: 0.2813046	total: 4m 12s	remaining: 1m 17s
766:	learn: 0.2810512	total: 4m 13s	r

903:	learn: 0.2668244	total: 4m 59s	remaining: 31.8s
904:	learn: 0.2666794	total: 5m	remaining: 31.5s
905:	learn: 0.2665523	total: 5m	remaining: 31.2s
906:	learn: 0.2664664	total: 5m	remaining: 30.8s
907:	learn: 0.2663062	total: 5m 1s	remaining: 30.5s
908:	learn: 0.2661926	total: 5m 1s	remaining: 30.2s
909:	learn: 0.2660427	total: 5m 1s	remaining: 29.9s
910:	learn: 0.2660057	total: 5m 2s	remaining: 29.5s
911:	learn: 0.2658501	total: 5m 2s	remaining: 29.2s
912:	learn: 0.2658193	total: 5m 3s	remaining: 28.9s
913:	learn: 0.2657237	total: 5m 3s	remaining: 28.5s
914:	learn: 0.2656025	total: 5m 3s	remaining: 28.2s
915:	learn: 0.2653914	total: 5m 4s	remaining: 27.9s
916:	learn: 0.2652559	total: 5m 4s	remaining: 27.6s
917:	learn: 0.2651434	total: 5m 4s	remaining: 27.2s
918:	learn: 0.2650380	total: 5m 5s	remaining: 26.9s
919:	learn: 0.2649440	total: 5m 5s	remaining: 26.6s
920:	learn: 0.2648616	total: 5m 6s	remaining: 26.2s
921:	learn: 0.2647777	total: 5m 6s	remaining: 25.9s
922:	learn: 0.264633

0.8881464242611404

In [23]:
from lightgbm import LGBMClassifier
estimator8 =LGBMClassifier()
score8 = cross_val_score(estimator8, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
score8


0.7706608861140307

In [3]:
model_names = ['Logistic Regression', 'Decision Trees Classifier', 'Random Forest Classifier', 'XGB Classifier', 'Gradient Boosting Classifier', 'K-Nearest Neighbor Classifier', 'Support Vector Machines','CatBoost','LGBM']
f1_train_scores = [score, score,score,score3, score4, score5,score6,score7,score8]
base_models = pd.DataFrame(np.around(f1_train_scores,3), model_names, columns=['F1_SCORES']).sort_values('F1_SCORES', ascending=False)
base_models

NameError: name 'score' is not defined

In [2]:
train_data_svm.describe()

NameError: name 'train_data_svm' is not defined

In [ ]:
#Before outliers removal
plt.subplot(3,2,1)
train_prep[box_features].boxplot()
plt.ylim([-15, 15])
plt.title('Before Outlier Removal', size=15)
plt.subplot(3,2,6)
train_data_svm[box_features].boxplot()
plt.title('After Outlier Removal using One-class SVM', size=15)

In [24]:

# estimators = [LogisticRegression(C=1e-3, solver='saga', max_iter=4000), DecisionTreeClassifier(), RandomForestClassifier(), XGBClassifier(), SVC(), GradientBoostingClassifier()]
# f1_train_scores = []
# i=1
# for estimator in estimators:
#   f1_score_i = cross_val_score(estimator, x_train_sam, y_train_sam.values.ravel(), cv=10, scoring='f1_weighted').mean()
#   f1_train_scores.append(f1_score_i)

#   print('---------------MODEL {} ACCURACY {}---------------'.format(i, f1_score_i))
#   i += 1
# #print(f1_train_scores)

In [25]:
# #Merging train data
# new_train = pd.concat((x_train_scaled, pd.DataFrame(y_train, columns=["Lithology_encoded"])), axis=1)

# #Randomly sampling data
# sampled_train = new_train.sample(n=819358, random_state=0)


# #Spliting training data
# x_train_sam = sampled_train.drop(["Lithology_encoded"], axis=1)
# y_train_sam = sampled_train["Lithology_encoded"]

# new_test = pd.concat((x_test_scaled, pd.DataFrame(y_test, columns=["Lithology_encoded"])), axis=1)

# #Randomly sampling data
# sampled_test = new_test.sample(n=120000, random_state=None)